In [277]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [278]:
os.listdir()

['month_stock.csv',
 'Untitled1.ipynb',
 '.DS_Store',
 'trade_train_set',
 'Untitled.ipynb',
 '2020MiraeAsset',
 'df.csv',
 'total_data.csv',
 '.ipynb_checkpoints']

In [279]:
data = pd.read_csv('./total_data.csv')
monthly = pd.read_csv('./month_stock.csv')
stocks = pd.read_csv('./trade_train_set/stocks.csv')
trade_train = pd.read_csv('./trade_train_set/trade_train.csv')

In [280]:
stocks = stocks.drop(['index'], axis=1)
trade_train = trade_train.drop(['Unnamed: 0'], axis=1)

trade_train[(trade_train['그룹번호'] == 'MAD01') & (trade_train['종목번호'] == 'A000020')]

,기준년월,그룹번호,그룹내고객수,종목번호,그룹내_매수여부,그룹내_매도여부,매수고객수,매도고객수,평균매수수량,평균매도수량,매수가격_중앙값,매도가격_중앙값


In [281]:
def num_one_history(df, grp):
    data = df[df['그룹번호'] == grp][['기준년월', '종목번호']]
    data['대분류'] = 0
    data['중분류'] = 0
    data['소분류'] = 0
    
    for i in range(data.shape[0]):
        data.iloc[i,2] = list(stocks[stocks['종목번호'] == list(data['종목번호'])[i]]['표준산업구분코드_대분류'].unique())[0]
        data.iloc[i,3] = list(stocks[stocks['종목번호'] == list(data['종목번호'])[i]]['표준산업구분코드_중분류'].unique())[0]
        data.iloc[i,4] = list(stocks[stocks['종목번호'] == list(data['종목번호'])[i]]['표준산업구분코드_소분류'].unique())[0]
    return data

In [282]:
adj = trade_train.sort_values(by=['기준년월', '그룹번호', '매수고객수'], ascending=[True, True, False]).groupby(['기준년월', '그룹번호']).head(3)

trade_train[trade_train['그룹번호'] == 'MAD07']

,기준년월,그룹번호,그룹내고객수,종목번호,그룹내_매수여부,그룹내_매도여부,매수고객수,매도고객수,평균매수수량,평균매도수량,매수가격_중앙값,매도가격_중앙값
342,201907,MAD07,86,A009150,Y,Y,4,3,45,33,91017,90500
1854,201908,MAD07,86,A093370,Y,Y,3,3,2539,2718,8956,10234
3262,201909,MAD07,86,A009150,Y,Y,3,6,11,64,97500,101377
4680,201910,MAD07,86,A215600,Y,Y,3,3,738,975,10450,13650
6340,201911,MAD07,86,A005930,Y,Y,5,4,60,64,52000,52200
...,...,...,...,...,...,...,...,...,...,...,...,...
26291,202006,MAD07,86,A090430,Y,Y,3,5,14,35,162500,175500
26292,202006,MAD07,86,A090460,Y,N,3,0,40,0,20100,-1
26293,202006,MAD07,86,A091990,Y,Y,3,4,78,121,93480,93144
26294,202006,MAD07,86,A096770,Y,Y,16,19,79,124,125500,127500


In [283]:
num_one_history(adj, 'MAD07')

,기준년월,종목번호,대분류,중분류,소분류
342,201907,A009150,제조업,통신장비 제조업,전자부품 제조업
1854,201908,A093370,제조업,화학물질 및 화학제품 제조업;,기초 화학물질 제조업
3262,201909,A009150,제조업,통신장비 제조업,전자부품 제조업
4680,201910,A215600,전문· 과학 및 기술 서비스업,연구개발업,자연과학 및 공학 연구개발업
6340,201911,A005930,제조업,통신장비 제조업,통신 및 방송 장비 제조업
6341,201911,A010060,제조업,화학물질 및 화학제품 제조업;,기초 화학물질 제조업
6343,201911,A068270,제조업,의료용 물질 및 의약품 제조업,기초 의약물질 및 생물학적 제제 제조업
8060,201912,A005930,제조업,통신장비 제조업,통신 및 방송 장비 제조업
8059,201912,A000660,제조업,전자부품· 컴퓨터· 영상· 음향 및,반도체 제조업
9601,202001,A005930,제조업,통신장비 제조업,통신 및 방송 장비 제조업


In [284]:
def freq(df):
    big = pd.DataFrame(df['대분류'].value_counts().nlargest(3)).reset_index(drop=False)
    big.columns = ['대분류', '카운트']
    mid = pd.DataFrame(df['중분류'].value_counts().nlargest(3)).reset_index(drop=False)
    mid.columns = ['중분류', '카운트']
    small = pd.DataFrame(df['소분류'].value_counts().nlargest(3)).reset_index(drop=False)
    small.columns = ['소분류', '카운트']

    
    print(big)
    print('\n')
    print(mid)
    print('\n')
    print(small)

freq(num_one_history(adj, 'MAD14'))

                     대분류  카운트
0                    제조업   24
1  전기· 가스· 증기 및 공기조절 공급업   12


                     중분류  카운트
0               통신장비 제조업   18
1  전기· 가스· 증기 및 공기조절 공급업   12
2         자동차 및 트레일러 제조업    4


                 소분류  카운트
0     통신 및 방송 장비 제조업   16
1                전기업   12
2  자동차용 엔진 및 자동차 제조업    4


In [285]:
adj = trade_train.sort_values(by=['기준년월', '그룹번호', '매수고객수'], ascending=[True, True, False]).groupby(['기준년월', '그룹번호']).head(3)

adj

,기준년월,그룹번호,그룹내고객수,종목번호,그룹내_매수여부,그룹내_매도여부,매수고객수,매도고객수,평균매수수량,평균매도수량,매수가격_중앙값,매도가격_중앙값
10,201907,MAD01,288,A066570,Y,Y,15,9,60,223,69900,76600
8,201907,MAD01,288,A009150,Y,Y,14,10,76,83,94211,94900
4,201907,MAD01,288,A005930,Y,Y,13,19,34,101,45350,46850
14,201907,MAD02,112,A042670,Y,Y,4,4,872,2340,5900,6025
12,201907,MAD02,112,A006800,Y,Y,3,3,1280,6639,7760,7895
...,...,...,...,...,...,...,...,...,...,...,...,...
30131,202006,MAD47,244,A017670,Y,Y,46,21,27,42,208000,221000
30123,202006,MAD47,244,A005935,Y,Y,37,18,77,100,46311,46752
30178,202006,MAD48,163,A035420,Y,Y,26,27,52,51,242500,240000
30179,202006,MAD48,163,A035720,Y,Y,24,24,18,23,267500,271500


In [286]:
aa = trade_train[['기준년월', '종목번호', '매수고객수']].groupby(['기준년월','종목번호']).sum().sort_values(
    by = ['기준년월','매수고객수'], ascending=[True, False]).reset_index(drop=False)

aa.groupby(['기준년월']).head(3)

,기준년월,종목번호,매수고객수
0,201907,A005930,487
1,201907,A093370,339
2,201907,A009150,319
376,201908,A005930,736
377,201908,A215600,437
378,201908,A000660,257
726,201909,A005930,576
727,201909,A000660,283
728,201909,A084990,185
1068,201910,A215600,686


In [287]:
'''****** 변수 수정

거래량, 가격 -> 1을 기준으로 scaling, or 전월을 기준으로 +a%
20년도 7월에 포함되지 않은 것은 모두 drop


****** 확실히 버릴 것
그룹내_매수여부
그룹내_매도여부
기준년월
평균매수수량
평균매도수량
매수가격_중앙값
매도가격_중앙값
20년 7월 top3
종목명


******* 애매한 것
매도고객수 (전월 데이터를 사용할까?) -> 사용한다면 19년 7월은 어떻게 할까?

산업 대분류

산업 중분류

산업 소분류



******* 가져갈 것.

그룹번호
그룹내 고객수
종목번호
매수고객수 (Y)
거래량
거래금액_만원단위(정규화)
시장구분(정규화)
평균가격(정규화)
직전달 TOP3 여부'''

'****** 변수 수정\n\n거래량, 가격 -> 1을 기준으로 scaling, or 전월을 기준으로 +a%\n20년도 7월에 포함되지 않은 것은 모두 drop\n\n\n****** 확실히 버릴 것\n그룹내_매수여부\n그룹내_매도여부\n기준년월\n평균매수수량\n평균매도수량\n매수가격_중앙값\n매도가격_중앙값\n20년 7월 top3\n종목명\n\n\n******* 애매한 것\n매도고객수 (전월 데이터를 사용할까?) -> 사용한다면 19년 7월은 어떻게 할까?\n\n산업 대분류\n\n산업 중분류\n\n산업 소분류\n\n\n\n******* 가져갈 것.\n\n그룹번호\n그룹내 고객수\n종목번호\n매수고객수 (Y)\n거래량\n거래금액_만원단위(정규화)\n시장구분(정규화)\n평균가격(정규화)\n직전달 TOP3 여부'

In [288]:
data = data[data['20년7월TOP3대상여부'] == 'Y']

In [289]:
data.columns

Index(['기준년월', '그룹번호', '그룹내고객수', '종목번호', '그룹내_매수여부', '그룹내_매도여부', '매수고객수',
       '매도고객수', '평균매수수량', '평균매도수량', '매수가격_중앙값', '매도가격_중앙값', '거래량', '거래금액_만원단위',
       '20년7월TOP3대상여부', '시장구분', '종목명', '표준산업구분코드_대분류', '표준산업구분코드_소분류',
       '표준산업구분코드_중분류', '평균가격'],
      dtype='object')

In [290]:
data = data.drop(['그룹내_매수여부', '그룹내_매도여부', '평균매수수량', '평균매도수량', '매수가격_중앙값', '매도가격_중앙값', 
          '20년7월TOP3대상여부', '종목명', '기준년월'], axis=1)

In [291]:
from sklearn.preprocessing import StandardScaler
data[['거래금액_만원단위', '평균가격']] = StandardScaler().fit_transform(data[['거래금액_만원단위', '평균가격']])

In [294]:
a = trade_train[['기준년월', '그룹번호', '종목번호', '매수고객수']].groupby(['기준년월', '그룹번호','종목번호']).sum(
).sort_values(by = ['기준년월','그룹번호','매수고객수'], ascending=[True, True, False]).reset_index(drop=False)

a = a.groupby(['기준년월', '그룹번호']).head(3)

prev_month_list = list(a[a['기준년월'] == 202006]['종목번호'])

data['직전달TOP3여부'] = 0
for i in range(data.shape[0]):
    if data.iloc[i,2] in prev_month_list:
        data.iloc[i,-1] = 1

In [296]:
sum(data['직전달TOP3여부'] == 1)

4805

In [297]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
def dummy(data,col):
    lab=LabelEncoder()
    aa=lab.fit_transform(data[col]).reshape(-1,1)
    ohe=OneHotEncoder(sparse=False)
    column_names=[col+'_'+ str(i) for i in lab.classes_]
    return(pd.DataFrame(ohe.fit_transform(aa),columns=column_names))

In [298]:
categ_columns = ['그룹번호', '종목번호', '표준산업구분코드_중분류', '표준산업구분코드_소분류']
n_columns = ['그룹내고객수', '매수고객수', '거래량', '거래금액_만원단위', '평균가격', '시장구분', '직전달TOP3여부']

df = data[n_columns]
df.index = range(df.shape[0])

lab = LabelEncoder()
for col in lab_cols:
    temp_df = lab.fit_transform(data[col]).reshape(-1,1)
    temp_df = pd.DataFrame(temp_df)
    temp_df.column = i
    df = pd.concat([df, temp_df], axis=1)

In [299]:
for col in categ_columns:
    temp_df = dummy(data, col)
    df = pd.concat([df, temp_df], axis=1)

In [300]:
lab = LabelEncoder()
df['시장구분'] = pd.DataFrame(lab.fit_transform(data['시장구분']).reshape(-1,1))
df['직전달TOP3여부'] = pd.DataFrame(lab.fit_transform(data['직전달TOP3여부']).reshape(-1,1))

In [301]:
df

,그룹내고객수,매수고객수,거래량,거래금액_만원단위,평균가격,시장구분,직전달TOP3여부,그룹번호_MAD01,그룹번호_MAD02,그룹번호_MAD03,...,표준산업구분코드_소분류_전자부품 제조업,표준산업구분코드_소분류_종합 소매업,표준산업구분코드_소분류_측정· 시험· 항해· 제어 및 기타 정밀기기 제조업; 광학기기 제외,표준산업구분코드_소분류_컴퓨터 프로그래밍· 시스템 통합 및 관리업,표준산업구분코드_소분류_텔레비전 방송업,표준산업구분코드_소분류_통신 및 방송 장비 제조업,표준산업구분코드_소분류_특수 목적용 기계 제조업,표준산업구분코드_소분류_플라스틱제품 제조업,표준산업구분코드_소분류_합성고무 및 플라스틱 물질 제조업,표준산업구분코드_소분류_항공 여객 운송업
0,288,7,2.993388e+06,0.229490,-0.111103,1,1,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,776,23,2.993388e+06,0.229490,-0.111103,1,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,890,3,2.993388e+06,0.229490,-0.111103,1,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1615,27,2.993388e+06,0.229490,-0.111103,1,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,634,7,2.993388e+06,0.229490,-0.111103,1,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7485,2879,6,1.061898e+05,-0.599464,0.013648,1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7486,2879,0,1.635518e+06,-0.611417,-0.750732,1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7487,2879,3,5.442303e+05,-0.624016,-0.734887,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7488,2879,6,1.758884e+05,-0.614776,-0.506165,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [302]:
df.to_csv('./df.csv',mode='w', header=True, index=False)